In [1]:
from config import *
from encoders import *
from data import *
from models import *
from evals import *
print("using device: ", device)

import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# for debugging
# import importlib
# import models
# importlib.reload(models)


/Users/joyqiu/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


using device:  cpu


# Ugly Engineering

In [4]:
# ugly engineering here
df = pd.read_csv('../../data/HR_events.csv')
df_y = pd.read_excel('../../data/PAS Challenge Outcome Data.xlsx', engine="calamine")
df_y = df_y[['VitalID', 'Died']]
df = df.merge(df_y, on='VitalID', how='left')
df['label'] = df.index.to_series()
df['text'] = df['Died'].apply(lambda x: 'This infant will die in 7 days. ' if x == 1 else 'This infant will survive. ')
# df['text'] = df['text'] +' '+ df['event_description'].astype(str)


df_train, df_test = train_test_split(df, test_size=0.2, stratify=df['text'])    
# df_train['text'] = df_train['text'] +' '+ df_train['event_description'].astype(str)
# df_test['text'] = df_test['text'] +' '+ df_test['event_description'].astype(str)



df_new = df_train
df_new['outcome'] = df_new['Died'].apply(lambda x: 'class1' if x == 1 else 'class2')
df_new_y = pd.get_dummies(df_new['outcome'])
y_true_train = torch.tensor(df_new_y.values)
ts_df_train = df_new.loc[:,'1':'300']


df_new = df_test
df_new['outcome'] = df_new['Died'].apply(lambda x: 'class1' if x == 1 else 'class2')
df_new_y = pd.get_dummies(df_new['outcome'])
y_true_test = torch.tensor(df_new_y.values)
ts_df_test = df_new.loc[:,'1':'300']

txt_ls = ['die', 'survive']

# Main Usage

In [9]:
# ---- config that need to be changed ----
overwrite = True
model_name = 'clip_hr_death_binary'
init_lr = 0.01 # initial learning rate
loss_type = 'similarity'


def check_data_ready():
    required_objects = {
        'y_true_train': (pd.DataFrame, np.ndarray, torch.Tensor),
        'ts_df_train': pd.DataFrame,
        'y_true_test': (pd.DataFrame, np.ndarray, torch.Tensor),
        'ts_df_test': pd.DataFrame,
        'txt_ls': list
    }
    
    for obj_name, expected_type in required_objects.items():
        # Check if object exists in globals
        if obj_name not in globals():
            raise ValueError(f"Missing required object: {obj_name}")
        
        obj = globals()[obj_name]
        # Check if object is None
        if obj is None:
            raise ValueError(f"Object {obj_name} is None")
            
        # Check type
        if not isinstance(obj, expected_type):
            raise TypeError(f"{obj_name} should be of type {expected_type}, but got {type(obj)}")
        
        # Check if empty
        if hasattr(obj, '__len__') and len(obj) == 0:
            raise ValueError(f"{obj_name} is empty")

    print("✓ All required objects are properly defined and non-empty")
    return True
try:
    check_data_ready()
except Exception as e:
    print(f"Error: {e}")
    print("Please define all required objects before proceeding")



✓ All required objects are properly defined and non-empty


In [10]:

# ---- ready dataloader ---- 
ts_f_train, tx_f_train, labels_train = get_features(df_train,ts_encoder_name,text_encoder_name)
train_dataloader = CLIPDataset(ts_f_train, tx_f_train, labels_train).dataloader(batch_size=batch_size)
ts_f_test, tx_f_test, labels_test = get_features(df_test, ts_encoder_name,text_encoder_name)
test_dataloader = CLIPDataset(ts_f_test, tx_f_test, labels_test).dataloader(batch_size=batch_size)

# ---- ready model ---- 
model_path = './results/'+model_name+'_'+loss_type+'.pth' 
eval_path = './results/'+model_name+'_'+loss_type+'_evals.pth'
# Initialize model
model = CLIPModel(
        ts_dim=ts_f_train.shape[1],    # 32
        text_dim=tx_f_train.shape[1],  # 768
        projection_dim=embedded_dim
    )
optimizer = torch.optim.AdamW(model.parameters(), lr=init_lr)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',           # Reduce LR when metric stops decreasing
    factor=0.9,          # Multiply LR by this factor
    patience=patience,          # Number of epochs to wait before reducing LR
    min_lr=1e-20         # Don't reduce LR below this value
)

/Users/joyqiu/anaconda3/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Layer (type:depth-idx)                   Param #
CLIPModel                                1
├─Sequential: 1-1                        --
│    └─Linear: 2-1                       4,224
│    └─LeakyReLU: 2-2                    --
│    └─Linear: 2-3                       33,024
│    └─LeakyReLU: 2-4                    --
│    └─Linear: 2-5                       32,896
│    └─LeakyReLU: 2-6                    --
│    └─Linear: 2-7                       16,512
├─Sequential: 1-2                        --
│    └─Linear: 2-8                       98,432
│    └─LeakyReLU: 2-9                    --
│    └─Linear: 2-10                      33,024
│    └─LeakyReLU: 2-11                   --
│    └─Linear: 2-12                      65,792
│    └─LeakyReLU: 2-13                   --
│    └─Linear: 2-14                      32,896
│    └─LeakyReLU: 2-15                   --
│    └─Linear: 2-16                      16,512
Total params: 333,313
Trainable params: 333,313
Non-trainable params: 0


In [ ]:
# ---- train and eval ---- 
if overwrite:
    # remove model_path, loss_path, eval_metrics_path
    if os.path.exists(model_path):
        os.remove(model_path)
    if os.path.exists(eval_path):
        os.remove(eval_path)
    overwrite = False # reset overwrite to False
if overwrite or not os.path.exists(model_path):
    for i in range(num_saves): 
        train_losses_tmp, test_losses_tmp = train(model, 
                                                train_dataloader,
                                                test_dataloader, 
                                                optimizer, 
                                                scheduler,
                                                num_epochs, 
                                                device,
                                                loss_type)
        train_losses = train_losses + train_losses_tmp
        test_losses = test_losses + test_losses_tmp
        # every num_epochs, evaluate the model
        train_eval_metrics = eval_model(model, y_true_train, ts_df_train, txt_ls, ts_encoder_name, text_encoder_name)
        test_eval_metrics = eval_model(model, y_true_test, ts_df_test, txt_ls, ts_encoder_name, text_encoder_name)
        train_eval_metrics_list.append(train_eval_metrics)
        test_eval_metrics_list.append(test_eval_metrics)
        # save model and losses
        torch.save(model.state_dict(), model_path)
        torch.save({
            'train_losses': train_losses,
            'test_losses': test_losses,
            'train_evals': train_eval_metrics_list,
            'test_evals': test_eval_metrics_list }, eval_path)
        if i % 10 == 0: # every 10 saves, evaluate the model
            eval_dict = torch.load(eval_path)
            eval_dict_eng = eng_eval_metrics(eval_dict, binary=True, plot=True)
    
else:
    model.load_state_dict(torch.load(model_path))
    eval_dict = torch.load(eval_path)
    eval_dict_eng = eng_eval_metrics(eval_dict, binary=True, plot_confusion_matrices=True)

Epoch [1/50]
	Training Loss: 538.832481
	Testing Loss: 538.653737
	Learning Rate: 0.010000000
Epoch [2/50]
	Training Loss: 540.031876
	Testing Loss: 539.848409
	Learning Rate: 0.010000000
Epoch [3/50]
	Training Loss: 540.093658
	Testing Loss: 539.565938
	Learning Rate: 0.010000000
Epoch [4/50]
	Training Loss: 540.074422
	Testing Loss: 539.671987
	Learning Rate: 0.010000000
Epoch [5/50]
	Training Loss: 539.936656
	Testing Loss: 539.429006
	Learning Rate: 0.010000000
Epoch [6/50]
	Training Loss: 539.820165
	Testing Loss: 539.565308
	Learning Rate: 0.010000000
Epoch [7/50]
	Training Loss: 539.601176
	Testing Loss: 539.388102
	Learning Rate: 0.010000000
Epoch [8/50]
	Training Loss: 539.724589
	Testing Loss: 539.413167
	Learning Rate: 0.010000000
Epoch [9/50]
	Training Loss: 539.740415
	Testing Loss: 539.443787
	Learning Rate: 0.010000000
Epoch [10/50]
	Training Loss: 539.670659
	Testing Loss: 539.400655
	Learning Rate: 0.010000000
Epoch [11/50]
	Training Loss: 539.696002
	Testing Loss: 539

/Users/joyqiu/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/joyqiu/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/joyqiu/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/joyqiu/anaconda3/l

Epoch [1/50]
	Training Loss: 539.882261
	Testing Loss: 539.634867
	Learning Rate: 0.010000000
Epoch [2/50]
	Training Loss: 539.955709
	Testing Loss: 539.675863
	Learning Rate: 0.010000000
Epoch [3/50]
	Training Loss: 539.915665
	Testing Loss: 539.717834
	Learning Rate: 0.010000000
Epoch [4/50]
	Training Loss: 539.970446
	Testing Loss: 539.691976
	Learning Rate: 0.010000000
Epoch [5/50]
	Training Loss: 539.953089
	Testing Loss: 539.712646
	Learning Rate: 0.010000000
Epoch [6/50]
	Training Loss: 539.976140
	Testing Loss: 539.680440
	Learning Rate: 0.010000000
Epoch [7/50]
	Training Loss: 539.963562
	Testing Loss: 539.672729
	Learning Rate: 0.010000000
Epoch [8/50]
	Training Loss: 540.005211
	Testing Loss: 539.675171
	Learning Rate: 0.010000000
Epoch [9/50]
	Training Loss: 539.962519
	Testing Loss: 539.709788
	Learning Rate: 0.010000000
Epoch [10/50]
	Training Loss: 539.991941
	Testing Loss: 539.640442
	Learning Rate: 0.010000000
Epoch [11/50]
	Training Loss: 539.973757
	Testing Loss: 539

/Users/joyqiu/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/joyqiu/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/joyqiu/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/joyqiu/anaconda3/l

Epoch [1/50]
	Training Loss: 540.077449
	Testing Loss: 539.763814
	Learning Rate: 0.010000000
Epoch [2/50]
	Training Loss: 540.083206
	Testing Loss: 539.750305
	Learning Rate: 0.009000000
Epoch [3/50]
	Training Loss: 540.088994
	Testing Loss: 539.746338
	Learning Rate: 0.009000000
Epoch [4/50]
	Training Loss: 540.095487
	Testing Loss: 539.772227
	Learning Rate: 0.009000000
Epoch [5/50]
	Training Loss: 540.098770
	Testing Loss: 539.726176
	Learning Rate: 0.009000000
Epoch [6/50]
	Training Loss: 540.087331
	Testing Loss: 539.769531
	Learning Rate: 0.009000000
Epoch [7/50]
	Training Loss: 540.103622
	Testing Loss: 539.772176
	Learning Rate: 0.009000000
Epoch [8/50]
	Training Loss: 540.107666
	Testing Loss: 539.758413
	Learning Rate: 0.009000000
Epoch [9/50]
	Training Loss: 540.098686
	Testing Loss: 539.734070
	Learning Rate: 0.009000000
Epoch [10/50]
	Training Loss: 540.088702
	Testing Loss: 539.742981
	Learning Rate: 0.009000000
Epoch [11/50]
	Training Loss: 540.096868
	Testing Loss: 539

...

Epoch [1/50]
	Training Loss: 547.464340
	Testing Loss: 532.963542
	Learning Rate: 0.000004111
Epoch [2/50]
	Training Loss: 547.465329
	Testing Loss: 532.963094
	Learning Rate: 0.000004111
Epoch [3/50]
	Training Loss: 547.464462
	Testing Loss: 532.962423
	Learning Rate: 0.000004111
Epoch [4/50]
	Training Loss: 547.463720
	Testing Loss: 532.961202
	Learning Rate: 0.000004111
Epoch [5/50]
	Training Loss: 547.465347
	Testing Loss: 532.956319
	Learning Rate: 0.000004111
Epoch [6/50]
	Training Loss: 547.463425
	Testing Loss: 532.962840
	Learning Rate: 0.000004111
Epoch [7/50]
	Training Loss: 547.465897
	Testing Loss: 532.955444
	Learning Rate: 0.000004111
Epoch [8/50]
	Training Loss: 547.462708
	Testing Loss: 532.960378
	Learning Rate: 0.000004111
Epoch [9/50]
	Training Loss: 547.464864
	Testing Loss: 532.954183
	Learning Rate: 0.000004111
Epoch [10/50]
	Training Loss: 547.463033
	Testing Loss: 532.959635
	Learning Rate: 0.000004111
Epoch [11/50]
	Training Loss: 547.464045
	Testing Loss: 532

/Users/joyqiu/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/joyqiu/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/joyqiu/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch [1/50]
	Training Loss: 547.464269
	Testing Loss: 532.960744
	Learning Rate: 0.000003700
Epoch [2/50]
	Training Loss: 547.465220
	Testing Loss: 532.953898
	Learning Rate: 0.000003700
Epoch [3/50]
	Training Loss: 547.466034
	Testing Loss: 532.960347
	Learning Rate: 0.000003700
Epoch [4/50]
	Training Loss: 547.464638
	Testing Loss: 532.956848
	Learning Rate: 0.000003700
Epoch [5/50]
	Training Loss: 547.465897
	Testing Loss: 532.957011
	Learning Rate: 0.000003700
Epoch [6/50]
	Training Loss: 547.465243
	Testing Loss: 532.960022
	Learning Rate: 0.000003700
Epoch [7/50]
	Training Loss: 547.465683
	Testing Loss: 532.956156
	Learning Rate: 0.000003700
Epoch [8/50]
	Training Loss: 547.466827
	Testing Loss: 532.958191
	Learning Rate: 0.000003700
Epoch [9/50]
	Training Loss: 547.464396
	Testing Loss: 532.961812
	Learning Rate: 0.000003700
Epoch [10/50]
	Training Loss: 547.465617
	Testing Loss: 532.955160
	Learning Rate: 0.000003700
Epoch [11/50]
	Training Loss: 547.466703
	Testing Loss: 532

/Users/joyqiu/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/joyqiu/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/joyqiu/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch [1/50]
	Training Loss: 547.466293
	Testing Loss: 532.956828
	Learning Rate: 0.000003700
Epoch [2/50]
	Training Loss: 547.467026
	Testing Loss: 532.952576
	Learning Rate: 0.000003700
Epoch [3/50]
	Training Loss: 547.466660
	Testing Loss: 532.954712
	Learning Rate: 0.000003700
Epoch [4/50]
	Training Loss: 547.467298
	Testing Loss: 532.954936
	Learning Rate: 0.000003700
Epoch [5/50]
	Training Loss: 547.467636
	Testing Loss: 532.949788
	Learning Rate: 0.000003700
Epoch [6/50]
	Training Loss: 547.465487
	Testing Loss: 532.955566
	Learning Rate: 0.000003700
Epoch [7/50]
	Training Loss: 547.466522
	Testing Loss: 532.951569
	Learning Rate: 0.000003700
Epoch [8/50]
	Training Loss: 547.468086
	Testing Loss: 532.949422
	Learning Rate: 0.000003700
Epoch [9/50]
	Training Loss: 547.464706
	Testing Loss: 532.957987
	Learning Rate: 0.000003700
Epoch [10/50]
	Training Loss: 547.466965
	Testing Loss: 532.945190
	Learning Rate: 0.000003700
Epoch [11/50]
	Training Loss: 547.468572
	Testing Loss: 532